# Labeling People in Drone Footage for Deep Learning 

This code will open each image in a file and display it for you to mark the people present. Start the mouse at the middle of a person, click and drag until the gray ellipses have completely encapsulated the person of interest. Release the mouse and move onto the next person. If you make a mistake, click the right mouse button. Ellipses in red are what will be saved to the file. Once you are done, click the enter button to remove all tracer and deleted ellipses (gray and white, respectively) and display only the final ellipses (red). Click the escape button to save the image as a labeled file and move onto the next image.

In [ ]:
# Load packages
import glob # for getting file names
import cv2 # for interactive drawing/labeling

In [ ]:
# List files
files = glob.glob('D:/Documents/SpringBoard/capstone-2/data/raw/all/*.png')

In [ ]:
# Set up for drawing ellipses
drawing = False # true if mouse is pressed
ix,iy = -1,-1 # initialize location
final_list = [] # initialize list to save ellipse locations

# Build the mouse call-back function
def draw_circle(event,x,y,flags,param):
    """ Creates an ellipse from a click and drag with the mouse """
    
    global ix,iy,drawing,mode, final_list # initialize global variables
       
    if event == cv2.EVENT_LBUTTONDOWN: # if you click the left mouse button
        drawing = True # you have started drawing
        ix,iy = x,y # save initial mouse locations

    elif event == cv2.EVENT_MOUSEMOVE: # if you move the mouse
        if drawing == True: # and you're drawing
            trace = cv2.ellipse(img,(ix,iy),(abs(x-ix),abs(y-iy)),0,0,360,(169,169,169),1)
                # build an ellipse on the image, with the center being at ix, iy, the radius
                # values for length and width being half of the distance your mouse has traveled
                # (current x location - initial x location), start at 0 angle, start at the 
                # ellipse being at 0 degrees and ending at 360 degrees, with a RGB color for 
                # gray and a line width of one

    elif event == cv2.EVENT_LBUTTONUP: # if you unclick the left mouse button
        drawing = False # you're no longer drawing
        cv2.ellipse(img,(ix,iy),(abs(x-ix),abs(y-iy)),0,0,360,(0,0,255),2)
            # add a final red ellipse to the ellipse you made last
        final_list.append((x,y,ix,iy)) # and add this to the final list
        
    elif event == cv2.EVENT_RBUTTONDOWN: # if you click the right mouse button
        if len(final_list) >= 1:  # if the final list has data in it
            cv2.ellipse(img,(final_list[-1][2],final_list[-1][3]),
                        (abs(final_list[-1][0]-final_list[-1][2]),
                         abs(final_list[-1][1]-final_list[-1][3])),
                        0,0,360,(255,255,255),2)
                # draw an ellipse over the most recent ellipse and make it white so that
                # you know it will be deleted
            del final_list[-1] # and remove it from the final list

In [ ]:
for file in files: # for each file that we need to label
    final_list = [] # start a new file list
    img = cv2.imread(file) # read in the file
    img = cv2.resize(img, (960, 540)) # rescale it so that it's easier to work with
    cv2.namedWindow('image') # name the window that the image will be produced in
    cv2.setMouseCallback('image',draw_circle) # set up mouse callbacks

    while(1): # while the window is open
        cv2.imshow('image',img) # show the image
        k = cv2.waitKey(1) & 0xFF # and set up a wait key to do things
        if k == 13: # if you hit enter (key 13)
            img = cv2.imread(file) # this will re-read the file
            img = cv2.resize(img, (960, 540)) # re-size the file
            cv2.namedWindow('image') # re-draw the window
            cv2.setMouseCallback('image',draw_circle) # set up mouse callbacks
            cv2.imshow('image',img) # show the image
            for shape in range(0,len(final_list)): # for each ellipse that has been saved
                cv2.ellipse(img,(final_list[shape][2],final_list[shape][3]),
                            (abs(final_list[shape][0]-final_list[shape][2]),
                             abs(final_list[shape][1]-final_list[shape][3])),
                            0,0,360,(0,0,255),2)
                    # plot the final shapes in red
        if k == 27: # if you hit the escape key
            filename = file.split('\\')[1] # grab the file name from the list of files
            filename = filename.split('.png')[0] # remove the .png
            filename = filename + '_labeled.png' # add labeled.png to the end
            cv2.imwrite(filename, img) # write the image to the file
            break # stop the interaction image

    cv2.destroyAllWindows() # and close all open windows